<a href="https://colab.research.google.com/github/Maraudier/SquadColaChat/blob/Maraudier-testing-1/BERTSQuADv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Obtain your Kaggle token. 
token = {"username":"lightshift","key":"b7ab47deb22bd26d7cb071a39c232d1c"}
#Colab and Kaggle set up
from google.colab import files
!pip install kaggle
!mkdir .kaggle
!mkdir ~/.kaggle
import json 
import os
if not os.path.exists('/content/.kaggle/kaggle.json'):
  with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
  !chmod 600 /content/.kaggle/kaggle.json
  !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
  !kaggle config set -n path -v{/content}

  !kaggle datasets download -d stanfordu/stanford-question-answering-dataset -p /content
  !unzip \*.zip
  !rm -rf /content/dataset/dataset/
  !ls

- path is now set to: {/content}
 57% 5.00M/8.73M [00:00<00:00, 15.3MB/s]
100% 8.73M/8.73M [00:00<00:00, 25.1MB/s]
Archive:  stanford-question-answering-dataset.zip
  inflating: dev-v1.1.json           
  inflating: train-v1.1.json         
dev-v1.1.json  stanford-question-answering-dataset.zip
sample_data    train-v1.1.json


In [0]:
#Set up GPU utility and print confirmation
import tensorflow as tf
import torch
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print('GPU confirmed: {}'.format(device_name))
else:
  raise SystemError("No GPU")

if torch.cuda.is_available():
  device = torch.device("cuda")
  print("%d GPU(s) " % torch.cuda.device_count())
  print("Using ", torch.cuda.get_device_name(0))

else:
  print("No GPU, defaulting to CPU")
  device = torch.device("cpu")

GPU confirmed: /device:GPU:0
1 GPU(s) 
Using  Tesla P100-PCIE-16GB


In [0]:
#Obtain inputs
import numpy as np
import pandas as pd
import json
from subprocess import check_output

In [0]:
#data management step 1
squad_df = pd.read_json("train-v1.1.json", orient='columns')
#Only 2 columns, data includes nested json. More work to organize data 
squad_df.head()

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1
3,"{'title': 'Genocide', 'paragraphs': [{'context...",1.1
4,"{'title': 'Antibiotics', 'paragraphs': [{'cont...",1.1


In [0]:
#Define a few functions to sort through the data further 
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'], verbose = 1):
  if verbose:
    print("Reading json...")
  file = json.loads(open(input_file_path).read())
  if verbose:
    print("Working...")
  js = pd.io.json.json_normalize(file , record_path )
  m = pd.io.json.json_normalize(file, record_path[:-1])
  r = pd.io.json.json_normalize(file,record_path[:-2])
  
  idx = np.repeat(r['context'].values, r.qas.str.len())
  ndx = np.repeat(m['id'].values, m['answers'].str.len())
  m['context'] = idx
  js['q_idx'] = ndx
  main = pd.concat([m[['id', 'question' , 'context']].set_index('id'),js.set_index('q_idx')], 1, sort=False).reset_index()
  main['c_id'] = main['context'].factorize()[0]
  if verbose:
    print("dataframe shape: {}".format(main.shape))
    print("Jobs Done")
  return main

In [0]:
##Another way to handle the json file
contexts = []
questions = []
answers_text = []
answers_start = []
for i in range(squad_df.shape[0]):
    topic = squad_df.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_text.append(q_a['answers'][0]['text'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            contexts.append(sub_para['context'])   
squad_df2 = pd.DataFrame({"context":contexts, 
                  "question": questions, 
                  "answer_start": answers_start, 
                  "text": answers_text})
squad_df2.head()

,context,question,answer_start,text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary


In [0]:
input_file_path = 'train-v1.1.json'
record_path = ['data', 'paragraphs', 'qas', 'answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path)

Reading json...
Working...
dataframe shape: (87599, 6)
Jobs Done


In [0]:
train.head()

,index,question,context,answer_start,text,c_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0


In [0]:
train['c_id'].describe()

count    87599.000000
mean      9477.616000
std       5495.417059
min          0.000000
25%       4903.000000
50%       9458.000000
75%      14245.000000
max      18890.000000
Name: c_id, dtype: float64

In [0]:
!pip install transformers

     |████████████████████████████████| 501kB 2.7MB/s 
     |████████████████████████████████| 870kB 12.3MB/s 
     |████████████████████████████████| 3.7MB 30kB/s 
     |████████████████████████████████| 1.0MB 40.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=51ae50ed39e517646f16da7fcce32a9794ef37e190f1977ab6d62b7f701a95ff
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

question = train[['question']].copy()
text = train[['text']].copy()
#input_ids = tokenizer.encode(question, text)
#token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
#start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))

#all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
#answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])

#assert answer == "asserted answer"

In [0]:
q_and_a = train[['question', 'text']].copy()
q_and_a.head()

,question,text
0,To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,What is in front of the Notre Dame Main Building?,a copper statue of Christ
2,The Basilica of the Sacred heart at Notre Dame...,the Main Building
3,What is the Grotto at Notre Dame?,a Marian place of prayer and reflection
4,What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary


In [0]:
#This Transformers package is a pytorch interface for working with BERT
!pip install transformers

In [0]:
from transformers import BertTokenizer
# bert-base-uncased
# 12-layer, 768-hidden, 12-heads, 110M parameters.
# Trained on lower-cased English text.

# bert-large-uncased
# 24-layer, 1024-hidden, 16-heads, 340M parameters.
# Trained on lower-cased English text.

# bert-base-cased
# 12-layer, 768-hidden, 12-heads, 110M parameters.
# Trained on cased English text.

# bert-large-cased
# 24-layer, 1024-hidden, 16-heads, 340M parameters.
# Trained on cased English text.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [0]:
#Visual Education
print("Original: ", sentences[0])
print("Tokenized: ", tokenizer.tokenize(sentences[0]))
print("Token IDs: ", tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

In [0]:
print(contexts[0])
print(tokenizer.tokenize(contexts[0])) 
print(tokenizer.tokenize(questions[0]))
print(tokenizer.tokenize(answers_text[0]))

In [0]:
squad_q_input_ids = []
for each_sent in questions:
  encoded = tokenizer.encode(each_sent, add_special_tokens = True)
  squad_q_input_ids.append(encoded)

print(questions[0])
print(squad_q_input_ids[0])

In [0]:
from keras.preprocessing.sequence import pad_sequences
# Combined below
# def set_input_id(collection):
#   input_ids = []
#   for sent in collection:
#     encoded_sentences = tokenizer.encode(sent, add_special_tokens = True)
#     input_ids.append(encoded_sentences)
#   return input_ids

# encode_qs = set_input_id(questions)
# encode_a =  set_input_id(answers_text)
# encode_c =  set_input_id(contexts)

def set_id_padding(collection):
  input_ids = []
  for sent in collection:
    encoded_sentences = tokenizer.encode(sent, add_special_tokens = True)
    input_ids.append(encoded_sentences)
  
  MAX_LENGTH = max([len(sen) for sen in input_ids])
  input_ids = pad_sequences(input_ids, 
                            maxlen=MAX_LENGTH, 
                            dtype="long", 
                            value=0, 
                            truncating="post", 
                            padding="post")
  return input_ids

pad_encode_qs = set_id_padding(questions)
pad_encode_a =  set_id_padding(answers_text)
#pad_encode_c =  set_id_padding(contexts)
print("Done")

In [0]:

###Left off Here Working on padding the q_a
def set_padding(collection):

  MAX_LENGTH = max([len(sen) for sen in collection])
  for sent in collection:
    padded = pad_sequences(sent,
                           maxlen=MAX_LENGTH, 
                           dtype="long", 
                           value=0, 
                           truncating="post", 
                           padding="post")
  return padded
pad_question = set_padding(questions)
pad_answers = set_padding(answers_text)

In [0]:
print(questions[0])
print(pad_answers_text[0])
print("#Encoded",encode_qs[0])
print("#Encoded and padded",pad_encode_qs[0])
print(answers_text[0])
print(pad_answers_text[0])
print("#Encoded",encode_a[0])
print("#Encoded and padded",pad_encode_a[0])

# print(contexts[0])
# print("#Encoded",encode_c[0])
# print("#Encoded",pad_encode_c[0])

In [0]:
#Cola encoding
input_ids = []
for each_sent in sentences:
  encoded_sentences = tokenizer.encode(each_sent, add_special_tokens = True) 
  input_ids.append(encoded_sentences)
#Visual Education
print('Original: ', sentences[0])
print('TokenIDs: ', input_ids[0])

In [0]:
#Add Padding for CoLA. Ensure size is equal across the board for all tensors. 
MAX_LENGTH = max([len(sen) for sen in input_ids])
input_ids = pad_sequences(input_ids, 
                          maxlen=MAX_LENGTH, 
                          dtype="long", 
                          value=0, 
                          truncating="post", 
                          padding="post")

In [0]:
#Atten masks helps ensure padding is recognized by the BERT model to be padding. 
attn_masks = []
for sent in input_ids:
  attn_mask = [int(token_id > 0) for token_id in sent]
  attn_masks.append(attn_mask)

In [0]:
#Train Test Split.
from sklearn.model_selection import train_test_split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
  input_ids, 
  label, 
  random_state=888, 
  test_size=.7)
#same for masks, and blanks because we cant use train, test, spit into 3
train_masks, val_masks, _, _ = train_test_split(
    attn_masks, 
    label, 
    random_state=888, 
    test_size=.7)

In [0]:
train_labels = torch.tensor(train_labels)
train_inputs = torch.tensor(train_inputs)
train_masks = torch.tensor(train_masks)

val_labels = torch.tensor(val_labels)
val_inputs = torch.tensor(val_inputs)
val_masks = torch.tensor(val_masks)

Revisit these areas below in more depth


In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 16 #No more than 32
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

Some model options:

    BertModel
    BertForMaskedLM
    BertForPreTraining
    BertForQuestionAnswering - SQuAD
    BertForTokenClassification
    BertForNextSentencePrediction - Chatbot user input
    BertForSequenceClassification - CoLA

In [0]:
from transformers import BertForQuestionAnswering, BertForSequenceClassification, AdamW, BertConfig
model = BertForSequenceClassification .from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False)
model.cuda()

In [0]:
modelQA = BertForQuestionAnswering.from_pretrained(
    "bert-base-uncased",
    num_labels = 4,
    output_attentions = False,
    output_hidden_states = False)
modelQA.cuda()

In [0]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5)
optimizer = Adam

In [0]:
from transformers import get_linear_schedule_with_warmup
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


In [0]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [0]:
import random
seed_val = 38
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
loss_values = []
for epoch_i in range(0, epochs):
  print("|-|")
  print('=Epoch {:}/{:}'.format(epoch_i + 1, epochs))
  print("Training...stand by")

  t0 = time.time()

  total_loss = 0

  model.train()

  for step, batch in enumerate(train_dataloader):
    if step % 40 == 0 and not step == 0:
      elapsed = format_time(time.time() - t0)
      print('Batch {:>5,} of {:>5,}. Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

    b_in_ids = batch[0].to(device)
    b_in_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    model.zero_grad()

    outputs = model(b_in_ids, 
                    token_type_ids=None, 
                    attention_mask=b_in_mask, 
                    labels=b_labels)
    
    loss = outputs[0]
    total_loss += loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
  avg_train_loss = total_loss / len(train_dataloader)

  loss_values.append(avg_train_loss)

  print("|-|")
  print("Avg train loss: {0:.2f}".format(avg_train_loss))


In [0]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(loss_values, 'b-o')

# Label the plot.
plt.title("Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.show()

In [0]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("./cola_public/raw/out_of_domain_dev.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

# Create sentence and label lists
sentences = df.sentence.values
labels = df.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end. hook
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(sent, add_special_tokens = True)    
    input_ids.append(encoded_sent)

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LENGTH, 
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


In [0]:

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')



In [0]:
print('Positive samples: %d of %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))

In [0]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

print('Calculating Matthews correlation coefficient for each batch. This is a measure of the quality of binary classifications.')

# For each input batch...
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)

In [0]:
matthews_set

In [0]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)


In [0]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

#torch.save(args, os.path.join(output_dir, 'training_args.bin'))


In [0]:
# # Mount Google Drive to this Notebook instance.
from google.colab import drive
drive.mount('/content/drive')
!cp -r ./model_save/ "./drive/Shared drives/BERT/"
# Load a trained model and vocabulary that you have fine-tuned
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

# # Copy the model to the GPU.
model.to(device)
